## Load test

In [ ]:
#Load test script to generate N concurrent requests to an API end point
import pandas
import asyncio
import aiohttp
import nest_asyncio


nest_asyncio.apply()

In [ ]:

def ping_llm():
    asyncio.run(main()) 


async def fetch(session, url, data):
    async with session.post(url, json=data) as response:
        return await response.text()

async def main():
    url = "https://google-generative-ai-samples-rk3ighzk6q-uc.a.run.app/embeddings"  # Replace with your actual URL
    num_requests = 10  # Number of parallel requests
    data = {"num1": 10, "num2": 5, "text": "['search_query: What is Liferay?', 'search_query: What is Google?']"}  # Sample POST data

    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url, data) for _ in range(num_requests)]
        responses = await asyncio.gather(*tasks)

    for response in responses:
        print(response)



In [ ]:

asyncio.run(main())

## For Vertex AI load test

In [ ]:
generation_config = {
    "max_output_tokens": 800,
    "top_p": 1,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
}

In [ ]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models

i=0

def generate(prompt="test"):
  vertexai.init(project="jsb-alto", location="us-central1")
  model = GenerativeModel(
    "gemini-1.0-pro-002",)

  vertexai.init(project="398507275014", location="us-west1")
  model = GenerativeModel("projects/398507275014/locations/us-central1/endpoints/2026705594223689728",)

  responses = model.generate_content(
      [prompt],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=False,
  )

  if responses.candidates:
        candidate = responses.candidates[0]
        # Check finish reason
        finish_reason = candidate.finish_reason
        if finish_reason == "RECITATION":
            print("Model retrieved information from a source.")
            # Access citation information if needed
            citation = candidate.citation_metadata.citations[0] if candidate.citation_metadata.citations else "No citation"
            print(citation)
            response = "Recitation"  # Or return a suitable default value
        elif candidate.content and candidate.content.parts:
            response = candidate.content.parts[0].text
            #print(response)
        else:
            print("No text content generated.")
            response = "No text content generated"
            #print("Prompt: ", prompt)
            #print("Model_Responses: ", responses)
  else:
      print("No content generated. Model returned an empty response.")
      response = ""


  #print("\n###",responses.text, end="")
  print("\n###",response, end="")

#generate()



In [ ]:
prompt="""{
  "messages": [
    {
      "role": "system",
      "content": "\n[INST] You are a helpful assistant. You specialize in extracting information from texts. Given a text file, your task is to parse and return the following fields a key value pair format. \n\nALWAYS REMEMBER:\n* While returning extraction, output the same prompts as keys.\n* Key is field, value is extracted output.\n* If any field below is not present, just put \"NA\" and DO NOT SKIP.\n* TAKE YOUR TIME AND MAKE SURE YOU EXTRACT THE CORRECT INFORMATION FOR EACH FIELD.  \n* DO NOT EXPLAIN, ONLY PROVIDE INFORMATION IN THE BELOW SCHEMA:\n\n\n- \"File Number\"\n- \"complete field value and next lines of insured section\"\n- \"complete field value and next lines of vested in section\"\n- \"The estate or interest in the land described herein and which is covered by this policy if vested in the named insured\"\n- \"Order Number\"\n- \"effective date\"\n- \"Amount of Insurance\"\n- \"policy identification number\"\n- \"Owner policy Amount of Insurance\"\n- \"Owner policy number\"\n- \"Owner Policy effective date\"\n- \"Loan policy number do not take the loan number\"\n- \"Loan Policy effective date\"\n- \"Loan policy Amount of Insurance\"\n- \"tax identification number\"\n- \"address referred to in the policy\"\n- \"property address referred to in the policy\"\n- \"property location\"\n- \"address reference\"\n- \"land referred to in the policy not legal description\"\n- \"street number\"\n- \"street name 1\"\n- \"street name 2\"\n- \"City name in united states of america\"\n- \"State name in united states of america\"\n- \"zipcode\"\n- \"County name in united states of america\"\n- \"Name of Insured only if its organisation name\"\n- \"Insured 1 human first name do not extract from title vested in\"\n- \"Insured 1 human middle name do not extract from title vested in\"\n- \"Insured 1 human last name do not extract from title vested in\"\n- \"Insured 1 human name suffix do not extract from title vested in do not extract last name\"\n- \"Insured 2 human first name do not extract from title vested in\"\n- \"Insured 2 human middle name do not extract from title vested in\"\n- \"Insured 2 human last name do not extract from title vested in\"\n- \"Insured 2 human name suffix do not extract from title vested in do not extract last name\"\n- \"buyer name on which title is vested in only if its organisation or bank name\"\n- \"buyer 1 human first name on which title is vested in\"\n- \"buyer 1 human middle name on which title is vested in\"\n- \"buyer 1 human last name on which title is vested in\"\n- \"buyer 1 human name suffix on which title is vested in do not extract last name\"\n- \"buyer 2 human first name on which title is vested in\"\n- \"buyer 2 human middle name on which title is vested in\"\n- \"buyer 2 human last name on which title is vested in\"\n- \"buyer 2 human name suffix on which title is vested in do not extract last name\"\n- \"seller name only if its organisation name\"\n- \"seller 1 human first name\"\n- \"seller 1 human middle name\"\n- \"seller 1 human last name\"\n- \"seller 1 human name suffix do not extract last name\"\n- \"seller 2 human first name\"\n- \"seller 2 human middle name\"\n- \"seller 2 human last name\"\n- \"seller 2 human name suffix do not extract last name\"\n- \"fa reference number\"\n- \"customer reference number\"\n- \"lender name from schedule A and if its organisation name only\"\n- \"lender 1 human first name from schedule A\"\n- \"lender 1 human middle name from schedule A\"\n- \"lender 1 human last name from schedule A\"\n- \"lender 1 human name suffix from schedule A do not extract last name\"\n- \"lender 2 human first name from schedule A\"\n- \"lender 2 human middle name from schedule A\"\n- \"lender 2 human last name from schedule A\"\n- \"lender 2 human name suffix from schedule A do not extract last name\"\n- \"Lot name or number\"\n- \"Unit name or number\"\n- \"Block name or number\"\n- \"subdivision name\"\n- \"book number\"\n- \"page number\"\n- \"range direction\"\n- \"section number\"\n- \"township direction\"\n- \"quarter section direction\"\n- \"Legal Description\"\n- \"Legal Description from Schedule A Continuation and next paragraphs of Legal Description\"\n- \"Legal Description from Schedule C and next paragraphs of Legal Description\"\n- \"Legal Description from exhibit A and next paragraphs of Legal Description\"\n"
    },
    {
      "role": "user",
      "content": "First American ATPS System Starter Document\nFirst American Title Insurance Company\n1650 West Big Beaver\nP.O. Box 1289\nTroy, MI 48099\n(800) 482-6162\nThis starter document contains information from Document No: 26785\nOrder Number:\nOrder Date:\nOrder Type:\nProperty Info:\n63-349732\n8/11/1994 8:37:00 AM\nMortgage Policy\nAddress:\nNot Provided\nCity:\nWATERFORD TOWNSHIP\nState:\nMI\nZip Code:\nCounty:\nNot Provided\nOAKLAND\nSubdivision: PART OF THE NE 1/4 OF SEC. 25, T3N., R9E WATERFORD TOWNSHIP\nPortion:\nSEE ATTACHED LEGAL\nTract:\nNot Provided\nLot:\nNot Provided\nUnit:\nNot Provided\nBook:\nNot Provided\nPage:\nNot Provided\nQ1-Q4:\nNot Provided\nArb:\nNot Provided\nSection:\n25\nRange:\nT3N R9E\nPolicy No:\nLP 2778302\nLiability:\n$8,000,000.00\nPolicy Type:\nLender's Policy\nParty(s):\n\u0422\u0443\u0440\u0435:\nNo:\nName:\nBuyer\n1\nLender\n1\nSUMMIT CROSSING SHOPPING CENTER\nNATIONWIDE LIFE INSURANCE CO. AND EMPLOYERS\nLIFE INS. CO.\nOther\n1\nALAN M. HURVITZ HONIGMAN, MILLER, SCHWARTZ\nAND COHN\nVesting:\nCurrent Vesting\nWINEMAN INVESTMENT CO., A MICHIGAN CORPORATION, AS TO AN UNDIVIDED\n50% INTEREST AND PONTIAC MALL LIMITED PARTNERSHIP, A MICHIGAN LIMITED\nPARTNERSHIP, AS TO AN UNDIVIDED 50% INTEREST\nProposed Vesting\nWINEMAN INVESTMENT CO., A MICHIGAN CORPORATION, AS TO AN UNDIVIDED\n50% INTEREST, AND PONTIAC MALL LIMITED PARTNERSHIP, A MICHIGAN\nLIMITED PARTNERSHIP, AS TO AN UNDIVIDED 50% INTEREST\nLender Vesting\nNATIONWIDE LIFE INSURANCE COMPANY, AN OHIO CORPORATION AND\nEMPLOYERS LIFE INSURANCE COMPANY OF WAUSAU, A WISCONSIN\nCORPORATION\nLegal Description:\nA PARCEL OF LAND IN THE NORTHEAST 1/4 OF SECTION 25, TOWN 3 NORTH,\nRANGE 9 EAST, WATERFORD TOWNSHIP, OAKLAND COUNTY, MICHIGAN,\nDESCRIBED AS FOLLOWS: BEGINNING ON THE NORTHERLY LINE OF ELIZABETH\nLAKE ROAD (120 FEET WIDE) AT A POINT DISTANT DUE WEST 2012.83 FEET ALONG\nTHE EAST AND WEST 1/4 LINE OF SECTION 25 AND NORTH 00 DEGREES 31\nMINUTES 40 SECONDS WEST, 60.00 FEET FROM THE EAST 1/4 CORNER OF SAID\nSECTION 25; THENCE NORTH 00 DEGREES 31 MINUTES 40 SECONDS WEST, 220.00\nFEET; THENCE DUE EAST 140.00 FEET; THENCE NORTH 00 DEGREES 31 MINUTES 40\nSECONDS WEST, 870.00 FEET; THENCE NORTH 15 DEGREES 51 MINUTES 40\nSECONDS WEST, 321.44 FEET; THENCE NORTH 00 DEGREES 31 MINUTES 40\nSECONDS WEST, 751.28 FEET; THENCE SOUTH 89 DEGREES 56 MINUTES 50\nSECONDS WEST, 700.62 FEET TO A POINT ON THE EAST LINE OF BIRDSLAND\nSUBDIVISION; THENCE ALONG SAID LINE SOUTH 00 DEGREES 26 MINUTES 35\nSECONDS EAST, 1278.63 FEET; THENCE DUE EAST, 350.03 FEET; THENCE SOUTH 00\nDEGREES 26 MINUTES 28 SECONDS EAST, 871.20 FEET TO A POINT ON THE\nNORTHERLY LINE OF ELIZABETH LAKE ROAD (120 FEET WIDE); THENCE ALONG\nSAID LINE DUE EAST 298.81 FEET TO THE POINT OF BEGINNING.\nEXCEPT THE FOLLOWING PARCEL DESCRIBED AS FOLLOWS:\nBEGINNING AT A POINT LOCATED DUE WEST 1872.83 FEET ALONG THE EAST AND\nWEST 1/4 LINE OF SECTION 25 AND NORTH 00 DEGREES 31 MINUTES 40 SECONDS\nWEST, 1011.11 FEET FROM THE EAST 1/4 CORNER OF SAID SECTION 25; THENCE\nNORTH 00 DEGREES 31 MINUTES 40 SECONDS WEST, 138.89 FEET; THENCE NORTH\n15 DEGREES 51 MINUTES 40 SECONDS WEST, 321.44 FEET; THENCE NORTH 00\nDEGREES 31 MINUTES 40 SECONDS WEST, 198.87 FEET; THENCE SOUTH 89\nDEGREES 33 MINUTES 25 SECONDS WEST, 38.14 FEET; THENCE SOUTH 00 DEGREES\n26 MINUTES 35 SECONDS EAST, 9.53 FEET; THENCE SOUTH 89 DEGREES 33\nMINUTES 25 SECONDS WEST, 241.32 FEET; THENCE SOUTH 00 DEGREES 26\nMINUTES 35 SECONDS EAST, 64.87 FEET; THENCE SOUTH 89 DEGREES 33 MINUTES\n25 SECONDS WEST, 21.00 FEET; THENCE SOUTH 00 DEGREES 26 MINUTES 35\nSECONDS EAST, 15.00 FEET; THENCE SOUTH 89 DEGREES 33 MINUTES 25 SECONDS\nWEST, 118.48 FEET; THENCE NORTH 00 DEGREES 26 MINUTES 35 SECONDS WEST,\n120.00 FEET; THENCE SOUTH 89 DEGREES 33 MINUTES 25 SECONDS WEST, 180.48\nFEET; THENCE NORTH 00 DEGREES 26 MINUTES 35 SECONDS WEST, 34.37 FEET;\nTHENCE SOUTH 89 DEGREES 33 MINUTES 25 SECONDS WEST, 102.00 FEET TO A\nPOINT ON THE EAST LINE OF BIRDSLAND SUBDIVISION; THENCE ALONG SAID\nLINE SOUTH 00 DEGREES 26 MINUTES 35 SECONDS EAST, 540.50 FEET; THENCE\nNORTH 89 DEGREES 33 MINUTES 25 SECONDS EAST, 193.00 FEET; THENCE NORTH\n00 DEGREES 26 MINUTES 35 SECONDS WEST, 35.50 FEET; THENCE NORTH 89\nDEGREES 33 MINUTES 25 SECONDS EAST, 205.23 FEET; THENCE SOUTH 00 DEGREES\n26 MINUTES 35 SECONDS EAST, 174.50 FEET; THENCE NORTH 89 DEGREES 33\nMINUTES 25 SECONDS EAST, 350.95 FEET; THENCE SOUTH 00 DEGREES 26 MINUTES\n35 SECONDS EAST, 33.10 FEET; THENCE NORTH 89 DEGREES 33 MINUTES 25\nSECONDS EAST, 38.20 FEET TO THE POINT OF BEGINNING.\nINCLUDING THE BENEFIT OF (i) THE ACCESS EASEMENT RECORDED IN LIBER 9750,\nPAGE 429 AND LIBER 9751, PAGE 195, (ii) THE EASEMENT FOR SUMMIT DRIVE\nRECORDED IN LIBER 10241, PAGE 085; LIBER 11721, PAGE 752 AND LIBER 12182,\nPAGE 564 AND (iii) THE EASEMENTS FOR PARKING AND ROADWAYS RECORDED IN\nLIBER 6644, PAGE 87, OAKLAND COUNTY RECORDS.\nAPN(s):\nTax Key 1:\nTax Key 2:\nNot Provided\n13-25-200-021\nOwner:\nNo Exceptions or Restrictions Provided\nLender:\nNo Exceptions or Restrictions Provided\nCommitment:\nALTA Loan Commitment (EXCEPTION)\n5.Mortgage between Wineman Investment Co., a Michigan Corporation, and Pontiac Mall\nLimited Partnership, a Michigan Limited Partnership, as mortgagor, and First of America Bank-\nSoutheast Michigan, National Association, as mortgagee in the original stated principal amount of\n$8,000,000.00, dated September 19, 1989, recorded on September 22, 1989 in Liber 11073, Page\n647 and amended by an Amendment to Mortgage dated October 3, 1991 and recorded on\nDecember 17, 1991 in Liber 12239, Page 618, together with Assignment of Rents and Leases and\nContracts of Sale dated September 19, 1989 and recorded on September 22, 1989 in Liber 11073,\nPage 641, OAKLAND County Records.\n6.Lease and terms and provisions thereof between Pontiac Mall Limited Partnership, a Michigan\nLimited Partnership and Wineman Investment Co., a Michigan Corporation, Landlord and Leaps\n& Bounds, Inc., a Delaware Corporation, Tenant, disclosed by Memorandum of Lease, dated June\n30, 1993, recorded on August 4, 1993 in Liber 13832, Page 773, OAKLAND County Records.\n7.Terms and conditions of Non-disturbance Attornment and Subordination Agreement by and\nbetween First of America Bank-Southeast Michigan, N.A. and Leaps and Bounds, Inc., a\nDelaware Corporation, recorded on August 4, 1993 in Liber 13832, Page 722, Oakland County\nRecords.\n8. Interest of Mile Fabrics of Akron North, Inc., Lessee, as disclosed in Memorandum of Lease\nrecorded on June 20, 1972 in Liber 5889, Page 878, Oakland County Records.\n9.Interest of Borman's Inc., lessee in an unrecorded lease dated July 27, 1988 wherein Pontiac\nMall Limited Partnership, a Michigan Limited Partnership, and Wineman Investment Co., a\nMichigan Corporation, were lessors, as set forth in Memorandum of Lease and Affidavit of\nInterest in Real Property, recorded on September 7, 1977 in Liber 10577, Page 048, Oakland\nCounty Records.\n10. Rights of tenants, as tenants only, now in possession of the land under unrecorded leases or\notherwise.\n11.Claim of Lien by Flag Fire Protection Company against Ramco-Gershenson, Inc., in the stated\namount of $9,700.00, dated August 26, 1993, recorded on August 26, 1993 in Liber 13905, Page\n043, OAKLAND County Records.\n12. Reservations of minerals, coal, oil and gas being on, within or under subject property, as set\nforth in Warranty Deed recorded on June 29, 1960 in Liber 4098, Page 597, Oakland County\nRecords.\n13.Terms and conditions of the Mutual Easements for parking areas and roadways as set forth in\nEasement recorded in Liber 6644, Page 87, Oakland County Records.\n14.Easements granted to the County of Oakland for sanitary sewer system purposes as set forth in\ninstruments recorded in Liber 8008, Page 629; Liber 8042, Page 570 and Liber 8042, Page 573,\nOakland County Records.\n15.Easement granted to the County of Oakland for water mains, as set forth in instruments\nrecorded in Liber 2821, Page 56, Liber 2973, Page 30 and Liber 4521, Page 705, Oakland County\nRecords.\n16.Consumers Power Company easement created by instrument recorded on December 18, 1972\nin Liber 6002, Page 303, OAKLAND County Records.\n17.Easement in favor of Consumers Power Company for pipeline purposes, as set forth in\ninstruments recorded in Liber 10409, Page 867 and Liber 10409, Page 872, Oakland County\nRecords.\n18.Easement granted to the Township of Waterford for sanitary sewers and water mains, as set\nforth in instrument recorded in Liber 6001, Page 208, Oakland County Records.\n19.Memorandum of Supplemental Agreement and the terms and conditions thereof recorded on\nFebruary 3, 1987 in Liber 9750, Page 453, OAKLAND County Records.\n20.Michigan Bell Telephone Company easement created by instrument recorded in Liber 11693,\nPage 264, OAKLAND County Records.\n21.Perpetual Access Agreement and the terms and conditions thereof dated November 24, 1987,\nrecorded on December 18, 1987 in Liber 10241, Page 085 and amended by Supplement to\nPerpetual Access Agreement dated January 10, 1991 and recorded on January 29, 1991 in Liber\n11721, Page 752, OAKLAND County Records.\n22.Easement Agreement and the terms and conditions thereof by and between Summit North\nLimited Partnership, a Michigan Limited Partnership, Wineman Investment Co., a Michigan\nCorporation, and Waterford Limited Dividend Housing Association Limited Partnership, a\nMichigan Limited Partnership, dated October 22, 1991, recorded on November 18, 1991 in Liber\n12182, Page 564, OAKLAND County Records.\n23.Building and Use Restrictions and other provisions but omitting restrictions, if any, based on\nrace, color, religion, or national origin in instruments recorded in Liber 9751, Page 195 and Liber\n9750, Page 429, OAKLAND County Records.\n24.Terms and conditions of Grant of Perpetual Access Easement executed by Pontiac Mall\nLimited Partnership, Wineman Investment Co. for benefit of the Dayton Hudson Corporation, a\nMinnesota Corporation, recorded on February 3, 1987 in Liber 9750, Page 429, Oakland County\nRecords.\n25.Terms and conditions of Reciprocal Easement Agreement executed by and between Pontiac\nMall Limited Partnership, a Michigan Limited Partnership, Wineman Investment Co., a Michigan\nCorporation and Dayton Hudson Corporation, a Minnesota Corporation, recorded on February 3,\n1987 in Liber 9751, Page 195, Oakland County Records.\n26.Easement for gas pipeline granted to Consumers Power Company recorded in Liber 10409,\nPage 855, Oakland County Records.\n27.The Company has been requested to include a Usury Endorsement with the Policy to be\nissued. A determination as to whether or not this coverage can be provided cannot be made until\nwe are furnished with a copy of the proposed note and mortgage.\n28. The Company has been requested to furnish a Subdivision Control Act Endorsement with the\nPolicy to be issued. This Endorsement cannot be provided at this time inasmuch as the land is not\nassessed as a separate parcel or parcels.\n29.Taxes and assessments not due and payable at Commitment Date.\nThe Policy to be issued will include the following Endorsements, copies of which are attached:\n(1) Zoning with Parking (Alta 3.1), Form No. 45-113.1 (2) Alta 9, Form No. 45-131 (3) Land\nDelineated on Survey, Form No. 45-193.1\nFSLegal 23647803\n"
    },
    {
      "role": "model",
      "content": 
    }
  ]
}
"""

In [ ]:
data_test = pandas.read_pickle("/Users/jasmeetbhatia/Documents/Gitlab/jsb-fsa-gitlab/faf-oss/dataset-500k/testInput_500K.pickle")
texts = data_test[1000:2000]

In [ ]:
import asyncio
from concurrent.futures import ThreadPoolExecutor

def llm_ping(text):
    # Your synchronous code here
    import time
    time.sleep(1)  # Simulating a synchronous operation
    print(f"Ping with {text}!")

async def run_load_test():
    loop = asyncio.get_running_loop()
    #texts = [f"text_{i}" for i in range(10)]  # Example texts to pass
    #print(texts)
    with ThreadPoolExecutor() as executor:
        tasks = [loop.run_in_executor(executor, generate, prompt) for prompt in texts]
        #tasks = [loop.run_in_executor(executor, generate, prompt) for _ in range(100)]
        await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(run_load_test())
